In [15]:
#GEOPANDAS
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [16]:
from shapely.geometry import Point, LineString, MultiPoint, MultiLineString, Polygon, MultiPolygon

In [3]:
#Quiz Considerando un radio de 1km, encuentra el COPPEL que tiene una mayor densidad de probes. Hint: Cambio el crs a mercator

In [17]:
pd_probe = pd.read_csv('data/probe.probe').sample(500)
pd_probe=pd_probe.rename(columns={'Unnamed: 0':'id'})
pd_probe=pd_probe[['id','timestamp','y','x','speed','provider']]
pd_probe['timestamp'] = pd.to_datetime(pd_probe['timestamp'],format= '%Y-%m-%dT%H:%M:%SZ')
pd_probe['timestamp']=pd_probe.timestamp.dt.tz_localize('UTC').dt.tz_convert('America/Mexico_City') 

hacerPoint = lambda row: Point(row.x,row.y)
pd_probe['geometry_probe'] = pd_probe.apply(hacerPoint,axis=1)
gdf_probe = gpd.GeoDataFrame(pd_probe,geometry='geometry_probe')
gdf_probe.crs = {'init' :'epsg:4326'}
gpd_states = gpd.read_file('data/Municipios/')
data_and_states = gpd.sjoin(gdf_probe, gpd_states, op='within')
gdf_probe=data_and_states[['id', 'timestamp','x','y','speed', 'provider','CVE_ENT','CVE_MUN','NOM_MUN']]#,'geometry_probe'
gdf_probe=gdf_probe.rename(columns={'CVE_ENT':'cve_ent'})
gdf_probe=gdf_probe.rename(columns={'CVE_MUN':'cve_mun'})

l = [Point(xy) for xy in zip(gdf_probe.y,gdf_probe.x)]
gdf_probe= gpd.GeoDataFrame(gdf_probe,geometry=l)
gdf_probe=gdf_probe.rename(columns={'geometry':'geometry_probe'})
gdf_probe.crs = {'init' :'epsg:4326'}
print(gdf_probe.size)
gdf_probe.head(3).reset_index()

4990


,index,id,timestamp,x,y,speed,provider,cve_ent,cve_mun,NOM_MUN,geometry_probe
0,250798,10336256,2017-03-30 15:13:27-06:00,-96.867306,19.545621,0.0,1,30,087,Xalapa,POINT (19.545621 -96.867306)
1,593966,4715165,2017-03-30 06:18:01-06:00,-100.107170,20.472670,75.0,2,22,012,Pedro Escobedo,POINT (20.47267 -100.10717)
2,376142,10405124,2017-03-30 15:16:34-06:00,-99.108800,19.379190,49.0,2,09,007,Iztapalapa,POINT (19.37919 -99.1088)


In [18]:
gdf_Coppel = pd.read_csv('data/Coppel.csv').sample(1000)

l = [Point(xy) for xy in zip(gdf_Coppel.latitud,gdf_Coppel.longitud)]
gdf_Coppel = gpd.GeoDataFrame(gdf_Coppel,geometry=l)
gdf_Coppel=gdf_Coppel.rename(columns={'Unnamed: 0':'id01'})
gdf_Coppel.crs = {'init' :'epsg:4326'}
print(gdf_Coppel.size)
gdf_Coppel.head(2).reset_index()

43000


,index,id01,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,...,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta,geometry
0,169,170,6707205,COPPEL,COPPEL SA DE CV,462210,Comercio al por menor en tiendas departamentales,51 a 100 personas,AVENIDA,CALLE 7,...,0109,23,NaN,FZAMUDIO@COPPEL.COM,WWW.COPPEL.COM,Fijo,19.412299,-99.057078,ENERO 2016,POINT (19.41229892 -99.05707789)
1,457,458,6391495,COPPEL HACIENDA 943,COPPEL SA DE CV,462210,Comercio al por menor en tiendas departamentales,31 a 50 personas,CALLE,PENSAMIENTO,...,1190,43,5.558351e+09,FZAMUDIO@COPPEL.COM,WWW.COPPEL.COM,Fijo,19.665034,-99.079837,JULIO 2010,POINT (19.66503372 -99.07983674)


In [19]:
#Ajusta a un formato de n digitos 
def FormatTxtEval(Val,Length): #formato de nuemero
    tmpVal=str(Val)
    lv=len(tmpVal)
    r=Length-lv
    #print(tmpVal.isnumeric() )
    if tmpVal.isnumeric() == True:
        if lv<=Length:
            l = ['0',tmpVal]
            n = [r, 1]
            Result=''.join([e * m for e, m in zip(l, n)])
        else:
            l = ['?',tmpVal]
            n = [Length, 1]
            Result=''.join([e * m for e, m in zip(l, n)])
    else:
        if lv<=Length:
            l = [' ',tmpVal]
            n = [r, 1]
            Result=''.join([e * m for e, m in zip(l, n)])
        else:
            l = ['?',tmpVal]
            n = [Length, 1]
            Result=''.join([e * m for e, m in zip(l, n)])
    return Result
FormatTxt = lambda v,n: FormatTxtEval(v,n)
#FormatTxtEval('12',5)

In [20]:
gdf_probe['cve_ent']=gdf_probe['cve_ent'].apply(FormatTxt,n=2)
gdf_probe['cve_mun']=gdf_probe['cve_mun'].apply(FormatTxt,n=3)
gdf_Coppel['cve_ent']=gdf_Coppel['cve_ent'].apply(FormatTxt,n=2)
gdf_Coppel['cve_mun']=gdf_Coppel['cve_mun'].apply(FormatTxt,n=3)
print(gdf_probe[['id','x','y','speed','provider','cve_ent','cve_mun']].head(2))
gdf_Coppel[['id01','id','nom_estab','cve_ent','cve_mun']].head(2)

              id           x          y  speed  provider cve_ent cve_mun
250798  10336256  -96.867306  19.545621    0.0         1      30     087
593966   4715165 -100.107170  20.472670   75.0         2      22     012


,id01,id,nom_estab,cve_ent,cve_mun
169,170,6707205,COPPEL,09,006
457,458,6391495,COPPEL HACIENDA 943,15,109


In [51]:
u_ProbeCoppel = pd.merge(gdf_probe,gdf_Coppel,how='left',on='cve_ent')
u_ProbeCoppel = u_ProbeCoppel.fillna(0)
print(u_ProbeCoppel.size)
print(len(u_ProbeCoppel.dropna(how='all')))
u_ProbeCoppel.head(1)

1438684
27667


,id_x,timestamp,x,y,speed,provider,cve_ent,cve_mun_x,NOM_MUN,geometry_probe,...,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta,geometry
0,10336256,2017-03-30 15:13:27-06:00,-96.867306,19.545621,0.0,1,30,087,Xalapa,POINT (19.545621 -96.867306),...,0237,4,9.242442e+09,FZAMUDIO@COPPEL.COM,WWW.COPPEL.COM,Fijo,17.432991,-95.024092,JULIO 2010,POINT (17.43299104 -95.02409176)


In [52]:
u_ProbeCoppel=u_ProbeCoppel.loc[(u_ProbeCoppel.latitud.astype(int)>0),:]
u_ProbeCoppel=u_ProbeCoppel.loc[(u_ProbeCoppel.cve_mun_x==u_ProbeCoppel.cve_mun_y),:].reset_index()
u_ProbeCoppel =u_ProbeCoppel[['index', 'id_x' ,'provider','x','y', 'cve_ent', 'cve_mun_x', \
                'NOM_MUN', 'id01', 'id_y', 'nom_estab', 'geometry_probe','latitud', 'longitud', 'geometry']]
print(u_ProbeCoppel.size)

34470


In [53]:
from geopy.distance import geodesic
def geodesic_dist(row):
    Pointsource=str(row['latitud'])+','+str(row['longitud'])
    Pointdest=str(row['y'])+','+str(row['x'])
    distancia =  (geodesic(Pointsource, Pointdest).km)
    return distancia

In [54]:
u_ProbeCoppel['Distancia']=u_ProbeCoppel.apply(geodesic_dist, axis=1)
u_ProbeCoppel=u_ProbeCoppel.loc[(u_ProbeCoppel.Distancia<=1),:].reset_index()
#u_ProbeCoppel.head(2)

In [56]:
CoppelMax=u_ProbeCoppel.loc[:,['nom_estab']].groupby(['nom_estab'])['nom_estab'].agg(['count']) \
            .sort_values('count', ascending=False).reset_index().head(10)
CoppelMax.rename(columns={'count':'probes_1km'})

,nom_estab,probes_1km
0,COPPEL MINITIENDA EL MOLINITO 850,2
1,COPPEL CANADA ZAPATERÍA SAN ESTEBAN 910,2
2,COPPEL CANADA ZAPATERÍA CÁRDENAS 596,2
3,COPPEL GRANJAS ESMERALDA 371,2
4,COPPEL PONTI 1098,2
5,COPPEL SAN ESTEBAN 1408,2
6,COPPEL CANADA 5 DE FEBRERO 702,2
7,COPPEL CANADA ZAPATERÍA TULYEHUALCO 558,2
8,COPPEL ERMITA IZTAPALAPA 455,1
9,COPPEL JUAREZ II 64,1


In [60]:
import requests
def driving_dist(row):
    Pointsource=str(row['longitud'])+','+str(row['latitud'])
    Pointdest=str(row['x'])+','+str(row['y'])
    url =  'http://router.project-osrm.org/route/v1/driving/'+Pointsource+';'+Pointdest
    payload = {"steps":"true","geometries":"geojson"}
    response = requests.get(url,params=payload)
    r = response.json()['routes']
    distancia = r[0]['distance']
#    duration = r[0]['duration']
#    ruta=LineString(r[0]['geometry']['coordinates'])
    return distancia/1000

In [61]:
u_ProbeCoppel['Dist_Driv']=u_ProbeCoppel.apply(driving_dist, axis=1)
u_ProbeCoppel.head(10)

,level_0,index,id_x,provider,x,y,cve_ent,cve_mun_x,NOM_MUN,id01,id_y,nom_estab,geometry_probe,latitud,longitud,geometry,Distancia,Dist_Driv
0,17,155,10154731,2,-99.112020,19.354650,09,007,Iztapalapa,187,6309676,COPPEL GRANJAS ESMERALDA 371,POINT (19.35465 -99.11202),19.356857,-99.116003,POINT (19.35685745 -99.11600332),0.484626,0.7204
1,21,189,10154731,2,-99.112020,19.354650,09,007,Iztapalapa,1116,6390006,COPPEL CANADA ZAPATERÍA TULYEHUALCO 558,POINT (19.35465 -99.11202),19.355966,-99.105849,POINT (19.35596623 -99.10584873000001),0.664563,1.0944
2,31,238,5950914,2,-99.048240,19.341920,09,007,Iztapalapa,1118,6308798,COPPEL ERMITA IZTAPALAPA 455,POINT (19.34192 -99.04824000000001),19.342480,-99.046206,POINT (19.34247959 -99.04620632),0.222486,0.5212
3,47,329,7568542,2,-99.077490,19.370800,09,007,Iztapalapa,169,6711720,COPPEL,POINT (19.3708 -99.07749),19.376610,-99.078061,POINT (19.37660972 -99.07806065),0.645904,0.8615
4,53,343,3209481,2,-99.112970,19.354230,09,007,Iztapalapa,187,6309676,COPPEL GRANJAS ESMERALDA 371,POINT (19.35423 -99.11297),19.356857,-99.116003,POINT (19.35685745 -99.11600332),0.431466,0.6267
5,57,377,3209481,2,-99.112970,19.354230,09,007,Iztapalapa,1116,6390006,COPPEL CANADA ZAPATERÍA TULYEHUALCO 558,POINT (19.35423 -99.11297),19.355966,-99.105849,POINT (19.35596623 -99.10584873000001),0.772498,1.9879
6,61,402,8623676,2,-98.980050,22.739000,28,021,El Mante,905,6405285,COPPEL CANADA ZAPATERÍA JUÁREZ 659,POINT (22.739 -98.98005000000001),22.744276,-98.974962,POINT (22.74427586 -98.97496231),0.783868,1.1933
7,123,1710,7816431,2,-99.174730,19.611090,15,109,Tultitl�n,446,6350339,COPPEL CHILPAN 251,POINT (19.61109 -99.17473),19.614156,-99.180764,POINT (19.61415585 -99.18076358),0.718179,0.8702
8,381,3113,8696718,1,-100.920428,22.123592,24,028,San Luis Potos�,1238,6389998,COPPEL CANADA ZAPATERÍA PLAZA SENDERO 553,POINT (22.123592 -100.920428),22.130588,-100.925551,POINT (22.13058805 -100.92555095),0.937810,2.1263
9,427,3923,5270920,2,-101.216450,19.696040,16,053,Morelia,569,6359712,COPPEL TRES PUENTES 216,POINT (19.69604 -101.21645),19.702238,-101.213020,POINT (19.70223788 -101.21301988),0.774647,1.0399
